https://post.naver.com/viewer/postView.nhn?volumeNo=29700625&memberNo=35871176&vType=VERTICAL

In [1]:
import os
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import matplotlib as mpl
import warnings
warnings.filterwarnings("ignore")

In [2]:
from lightgbm import LGBMClassifier

In [3]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, LabelEncoder

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [5]:
!unzip -qq "/content/gdrive/MyDrive/open (5).zip"

In [6]:
import pandas as pd

In [7]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [8]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [9]:
# X는 독립변수이므로 종속변수를 제거합니다. 또한 target 이외의 문자열 데이터를 제거합니다.
X = train.drop(["ID", "genre",'key'], axis = 1)
# y는 종속변수로 값을 설정합니다.
y = train[['genre']]

# train에서와 마찬가지로 문자열이 포함된 특성은 제거합니다.
test = test.drop(["ID",'key'], axis = 1)

In [10]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2)

#non

In [11]:
dtc = LGBMClassifier()
dtc.fit(X_train, y_train)
val_pred = dtc.predict(X_test)

# Macro f1 score을 사용하기 위해 average 인자 값을 "macro" 로 설정해줍니다.
print("현재 Macro F1 Score의 검증 점수는 {}입니다.".format(f1_score(val_pred, y_test, average = "macro")))

현재 Macro F1 Score의 검증 점수는 0.6440775816584019입니다.


non - 0.6480054006567213

#random search
Random Search는 주어진 범위 내에서 임의의 조합을 추출하여 최적의 조합을 탐색하는 방법입니다.

In [12]:
from sklearn.model_selection import RandomizedSearchCV

In [13]:
random_search = { 'max_depth' : [2,3,4,6,8,10,12,15,20],
                 'lambda ' : [0.1,0.3,0.6],
                 'num_leaves' : [1,2,3,4,5,6,7,8,9,10],
          'min_samples_leaf' :[1,2,4,6,8,10,20,30],
          'min_samples_split':[1,2,3,4,5,6,8,10]
          }
clf = LGBMClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
                               cv = 5, verbose= 1, random_state= 101, n_jobs = -1)
model.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=-1,
                   param_distributions={'lambda ': [0.1, 0.3, 0.6],
                                        'max_depth': [2, 3, 4, 6, 8, 10, 12, 15,
                                                      20],
                                        'min_samples_leaf': [1, 2, 4, 6, 8, 10,
                                                             20, 30],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              8, 10],
                                        'num_leaves': [1, 2, 3, 4, 5, 6, 7, 8,
                                                       9, 10]},
                   random_state=101, verbose=1)

In [14]:
val_pred = model.predict(X_test)

# Macro f1 score을 사용하기 위해 average 인자 값을 "macro" 로 설정해줍니다.
print("현재 Macro F1 Score의 검증 점수는 {}입니다.".format(f1_score(val_pred, y_test, average = "macro")))

현재 Macro F1 Score의 검증 점수는 0.6432016168747717입니다.


In [15]:
random_search = {'max_depth' : [2,3,4,6,8,10,12,15,20,100],
    'n_estimators': [50, 100, 150],
    'lambda ' : [0.1,0.3,0.6,0.7],
    'learning_rate': [0.01, 0.1, 0.5],
    'min_samples_leaf' :[1,2,4,6,8,10,20,30],
    'min_samples_split':[1,2,3,4,5,6,8,10],
    'num_leaves': [20,31, 61, 91],
    'subsample': [0.6, 0.8, 1.0],
}

clf = LGBMClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
                               cv = 5, verbose= 1, random_state= 101, n_jobs = -1)
model.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=-1,
                   param_distributions={'lambda ': [0.1, 0.3, 0.6, 0.7],
                                        'learning_rate': [0.01, 0.1, 0.5],
                                        'max_depth': [2, 3, 4, 6, 8, 10, 12, 15,
                                                      20, 100],
                                        'min_samples_leaf': [1, 2, 4, 6, 8, 10,
                                                             20, 30],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              8, 10],
                                        'n_estimators': [50, 100, 150],
                                        'num_leaves': [20, 31, 61, 91],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=101, verbose=1)

In [16]:
val_pred = model.predict(X_test)

# Macro f1 score을 사용하기 위해 average 인자 값을 "macro" 로 설정해줍니다.
print("현재 Macro F1 Score의 검증 점수는 {}입니다.".format(f1_score(val_pred, y_test, average = "macro")))

현재 Macro F1 Score의 검증 점수는 0.6444550172828037입니다.


In [17]:
print("Best parameters: ", model.best_params_)
print("Best score: ", model.best_score_)

Best parameters:  {'subsample': 0.6, 'num_leaves': 31, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 20, 'max_depth': 12, 'learning_rate': 0.1, 'lambda ': 0.6}
Best score:  0.6806856322422302


현재 Macro F1 Score의 검증 점수는 0.6448238804313243입니다.

약간 향상되었는데...

별 차이가 없다.

In [18]:
from sklearn.metrics import make_scorer, f1_score

In [19]:
# Define the scoring function
f1_scorer = make_scorer(f1_score, average='macro')

# define the parameter space for the LGBMClassifier
params = {'max_depth' : [2,3,4,6,8,10,12,15,20,100],
    'n_estimators': [50, 100, 150],
    'lambda ' : [0.1,0.3,0.6,0.7],
    'learning_rate': [0.01, 0.1, 0.5],
    'min_samples_leaf' :[1,2,4,6,8,10,20,30],
    'min_samples_split':[1,2,3,4,5,6,8,10],
    'num_leaves': [20,31, 61, 91],
    'subsample': [0.6, 0.8, 1.0],
    'min_data_in_leaf' : [20,25,30,15,10],
    'boosting_type': ['gbdt', 'dart']
}


# create an instance of the LGBMClassifier
lgbm = LGBMClassifier()

# create an instance of the RandomizedSearchCV
random_search = RandomizedSearchCV(lgbm, param_distributions=params, n_iter=10, cv=5, 
                                   scoring=f1_scorer, n_jobs=-1, random_state=42)

# fit the random search to the data
random_search.fit(X_train,y_train)

# print the best parameters and the best score
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

Best parameters:  {'subsample': 0.8, 'num_leaves': 91, 'n_estimators': 50, 'min_samples_split': 4, 'min_samples_leaf': 6, 'min_data_in_leaf': 25, 'max_depth': 12, 'learning_rate': 0.1, 'lambda ': 0.1, 'boosting_type': 'dart'}
Best score:  0.6359797545915786


In [20]:
val_pred = random_search.predict(X_test)

# Macro f1 score을 사용하기 위해 average 인자 값을 "macro" 로 설정해줍니다.
print("현재 Macro F1 Score의 검증 점수는 {}입니다.".format(f1_score(val_pred, y_test, average = "macro")))

현재 Macro F1 Score의 검증 점수는 0.635299718490676입니다.


#optuna

In [21]:
!pip install optuna


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.5 MB/s eta 0:00:00


In [22]:
import optuna

min_samples_leaf

In [23]:
def objective(trial):
    # Define the hyperparameter space
    num_leaves = trial.suggest_int('num_leaves', 10, 100)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.2)
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    subsample = trial.suggest_uniform('subsample', 0.5, 1)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.5, 1)
   
    # Initialize the LGBMClassifier
    lgbm = LGBMClassifier(num_leaves=num_leaves,
                           learning_rate=learning_rate,
                           n_estimators=n_estimators,
                           min_child_weight=min_child_weight,
                           subsample=subsample,
                           colsample_bytree=colsample_bytree,
                           random_state=42)
    
    # Fit the model
    lgbm.fit(X_train, y_train)
    
    # Compute F1-score on the validation set
    y_pred = lgbm.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    return 1 - f1

study = optuna.create_study()
study.optimize(objective, n_trials=100, n_jobs=-1)

# Print the best hyperparameters
print(study.best_params)

[I 2023-01-29 10:01:12,927] A new study created in memory with name: no-name-a24e7dae-73d0-41e0-8b47-b2533edd8484
[I 2023-01-29 10:01:51,261] Trial 0 finished with value: 0.3704750874766872 and parameters: {'num_leaves': 40, 'learning_rate': 0.15313747629838179, 'n_estimators': 175, 'min_child_weight': 8, 'subsample': 0.9578811089287604, 'colsample_bytree': 0.6616748005988594}. Best is trial 0 with value: 0.3704750874766872.
[I 2023-01-29 10:02:03,374] Trial 2 finished with value: 0.36464479724560805 and parameters: {'num_leaves': 35, 'learning_rate': 0.17375776580530106, 'n_estimators': 72, 'min_child_weight': 5, 'subsample': 0.5968897130394315, 'colsample_bytree': 0.9674830239695693}. Best is trial 2 with value: 0.36464479724560805.
[I 2023-01-29 10:02:31,907] Trial 3 finished with value: 0.3656394991281581 and parameters: {'num_leaves': 65, 'learning_rate': 0.046878003432727695, 'n_estimators': 126, 'min_child_weight': 4, 'subsample': 0.5485669939100994, 'colsample_bytree': 0.849879

{'num_leaves': 18, 'learning_rate': 0.04760875922391331, 'n_estimators': 405, 'min_child_weight': 5, 'subsample': 0.7980400086385043, 'colsample_bytree': 0.9019510700051276}


{'num_leaves': 11, 'learning_rate': 0.11226260952429802, 'n_estimators': 144, 'min_child_weight': 6, 'subsample': 0.9630753141870189, 'colsample_bytree': 0.5987133594396745}

In [28]:
params = {'num_leaves': 18, 'learning_rate': 0.04760875922391331, 'n_estimators': 405, 'min_child_weight': 5, 'subsample': 0.7980400086385043, 'colsample_bytree': 0.9019510700051276}

In [29]:
dtc = LGBMClassifier(**params)
dtc.fit(X_train, y_train)
val_pred = dtc.predict(X_test)

# Macro f1 score을 사용하기 위해 average 인자 값을 "macro" 로 설정해줍니다.
print("현재 Macro F1 Score의 검증 점수는 {}입니다.".format(f1_score(val_pred, y_test, average = "macro")))

현재 Macro F1 Score의 검증 점수는 0.6480161109492533입니다.


random-search Best parameters: {'subsample': 0.6, 'num_leaves': 31, 'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 10, 'learning_rate': 0.1, 'lambda ': 0.1} Best score: 0.6820638269831646

{'num_leaves': 29, 'learning_rate': 0.07482817047725152, 'max_depth': 3, 'n_estimators': 650, 'min_child_weight': 1, 'subsample': 0.9466836218824469, 'min_data_in_leaf': 105, 'colsample_bytree': 0.8561387391594382}

In [26]:
from optuna.integration import lightgbm as lgb

In [30]:
import optuna
from sklearn.datasets import load_iris
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier


def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 128),
        'min_samples_split': trial.suggest_uniform('min_samples_split', 0.01, 0.5),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-5, 10.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.0),
        'max_depth': trial.suggest_int('max_depth', -1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.01, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 20),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.01, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-8, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.01, 1.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
         'objective': 'multiclass',
    }
    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -1 * f1_score(y_test, y_pred, average='macro')

study = optuna.create_study()
study.optimize(objective, n_trials=100)

print("Best trial:")
trial = study.best_trial
print("Value: ", -1*trial.value)
print("Params: ", trial.params)
# Print the best hyperparameters
print(study.best_params)

[I 2023-01-29 10:52:54,852] A new study created in memory with name: no-name-dc85d5f8-3660-4d1f-83e2-ab6cc2948ed6
[I 2023-01-29 10:52:59,831] Trial 0 finished with value: -0.6408695155326837 and parameters: {'num_leaves': 78, 'min_samples_split': 0.24239090950065395, 'boosting_type': 'gbdt', 'reg_alpha': 0.00031533266249159086, 'min_child_weight': 0.00019851000482688503, 'learning_rate': 0.28508470521667045, 'max_depth': 3, 'n_estimators': 111, 'min_child_samples': 62, 'subsample': 0.6201236612341858, 'min_data_in_leaf': 11, 'colsample_bytree': 0.6500201991942889, 'reg_lambda': 3.491057859860046e-08, 'min_split_gain': 0.005163637406448774, 'bagging_fraction': 0.6807623557419571, 'feature_fraction': 0.9759812694819635}. Best is trial 0 with value: -0.6408695155326837.
[I 2023-01-29 10:54:06,785] Trial 1 finished with value: -0.5197437571857334 and parameters: {'num_leaves': 68, 'min_samples_split': 0.08387159939327168, 'boosting_type': 'gbdt', 'reg_alpha': 0.001953806415617467, 'min_chi

Best trial:
Value:  0.6547593427477406
Params:  {'num_leaves': 6, 'min_samples_split': 0.023876639700255442, 'boosting_type': 'gbdt', 'reg_alpha': 2.04295722199648e-05, 'min_child_weight': 0.0008298400717774186, 'learning_rate': 0.11693254825590113, 'max_depth': -1, 'n_estimators': 676, 'min_child_samples': 32, 'subsample': 0.43223926721122186, 'min_data_in_leaf': 1, 'colsample_bytree': 0.4504681826870456, 'reg_lambda': 2.4012299446894634e-06, 'min_split_gain': 0.07211265162133412, 'bagging_fraction': 0.8499810791528031, 'feature_fraction': 0.8961190726783418}
{'num_leaves': 6, 'min_samples_split': 0.023876639700255442, 'boosting_type': 'gbdt', 'reg_alpha': 2.04295722199648e-05, 'min_child_weight': 0.0008298400717774186, 'learning_rate': 0.11693254825590113, 'max_depth': -1, 'n_estimators': 676, 'min_child_samples': 32, 'subsample': 0.43223926721122186, 'min_data_in_leaf': 1, 'colsample_bytree': 0.4504681826870456, 'reg_lambda': 2.4012299446894634e-06, 'min_split_gain': 0.0721126516213

 0.642127725596212

In [31]:
params = {'num_leaves': 6, 'min_samples_split': 0.023876639700255442, 'boosting_type': 'gbdt', 'reg_alpha': 2.04295722199648e-05, 'min_child_weight': 0.0008298400717774186, 'learning_rate': 0.11693254825590113, 'max_depth': -1, 'n_estimators': 676, 'min_child_samples': 32, 'subsample': 0.43223926721122186, 'min_data_in_leaf': 1, 'colsample_bytree': 0.4504681826870456, 'reg_lambda': 2.4012299446894634e-06, 'min_split_gain': 0.07211265162133412, 'bagging_fraction': 0.8499810791528031, 'feature_fraction': 0.8961190726783418}

In [32]:
dtc = LGBMClassifier(**params)
dtc.fit(X_train, y_train)
val_pred = dtc.predict(X_test)

# Macro f1 score을 사용하기 위해 average 인자 값을 "macro" 로 설정해줍니다.
print("현재 Macro F1 Score의 검증 점수는 {}입니다.".format(f1_score(val_pred, y_test, average = "macro")))

현재 Macro F1 Score의 검증 점수는 0.6547593427477406입니다.


In [35]:
pred = dtc.predict(test)

In [36]:
submission = pd.read_csv("./sample_submission.csv")
submission["genre"] = pred

In [37]:
submission

,ID,genre
0,TEST_00000,Hiphop
1,TEST_00001,Underground Rap
2,TEST_00002,Emo
3,TEST_00003,techhouse
4,TEST_00004,Pop
...,...,...
16917,TEST_16917,techhouse
16918,TEST_16918,Hiphop
16919,TEST_16919,hardstyle
16920,TEST_16920,psytrance


In [38]:
submission.to_csv("./light-optuna final.csv", index = False)